In [143]:
!pip install pyclipper
!pip install shapely

import onnxruntime as ort
import numpy as np
from urllib.request import urlopen
import cv2
from shapely.geometry import Polygon
import pyclipper
import math


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [144]:
def norm(img):
    shape = (1, 1, 3)
    scale = 1.0 / 255.0
    std = np.array([0.229, 0.224, 0.225]).reshape(shape).astype("float32")
    mean = np.array([0.485, 0.456, 0.406]).reshape(shape).astype("float32")

    return (img.astype("float32") * scale - mean) / std

def image_padding(im):
        h, w, c = im.shape
        im_pad = np.zeros((640, 640, c), np.uint8)
        im_pad[:h, :w, :] = im
        return im_pad

def get_mini_boxes(contour):
        bounding_box = cv2.minAreaRect(contour)
        p = list(cv2.boxPoints(bounding_box))
        # print("before", p)
        points = sorted(p, key=lambda x: x[0])
        # print("after", p)
    
        index_1, index_2, index_3, index_4 = 0, 1, 2, 3
        if points[1][1] > points[0][1]:
            index_1 = 0
            index_4 = 1
        else:
            index_1 = 1
            index_4 = 0
        if points[3][1] > points[2][1]:
            index_2 = 2
            index_3 = 3
        else:
            index_2 = 3
            index_3 = 2

        box = [points[index_1], points[index_2], points[index_3], points[index_4]]
        return np.array(box), min(bounding_box[1])

def unclip(box, unclip_ratio):
        poly = Polygon(box)

        if poly.length <= 0:
            return None
            
        distance = poly.area * unclip_ratio / poly.length
        offset = pyclipper.PyclipperOffset()
        offset.AddPath(box, pyclipper.JT_ROUND, pyclipper.ET_CLOSEDPOLYGON)
        expanded = offset.Execute(distance)
        return expanded

def draw_text_det_res(dt_boxes, img):
    for box in dt_boxes:
        box = np.array(box).astype(np.int32).reshape(-1, 2)
        cv2.polylines(img, [box], True, color=(255, 255, 0), thickness=2)
    return img

def min_x_y_point(points):
    x = (math.inf, -math.inf)
    y = (math.inf, -math.inf)
    for p in points:
        x = (min(x[0], p[0]), max(x[1], p[0]))
        y = (min(y[0], p[1]), max(y[1], p[1]))

    return x, y
    

In [145]:
sess = ort.InferenceSession(
                "../model/det_onnx/model.onnx",
                providers=["CPUExecutionProvider"],
                sess_options=None,
            )
inputs = sess.get_inputs()

print(inputs[0], inputs[0].name, type(inputs[0].shape[2:][0]))

path = "https://i.ytimg.com/vi/srrlq7mdjEo/sddefault.jpg" 
# path = 'http://localhost:8000/dataset/card_1.jpg'
v = urlopen(path)

image_np = np.asarray(bytearray(v.read()), dtype="uint8")
# Decode the image using OpenCV
raw_img = cv2.imdecode(image_np, cv2.IMREAD_COLOR)
raw_img = cv2.resize(raw_img, (640, 640))

print(raw_img.shape)

padded_img = image_padding(raw_img)
print("image padded", padded_img.shape)

expected_img = np.random.uniform(0, 255, [640, 640, 3]).astype(np.uint8)
print("expected img shape", expected_img.shape)

norm_img = norm(padded_img)

transposed_img = norm_img.transpose((2, 0, 1))
print("transposed img", transposed_img.shape)

img = np.expand_dims(transposed_img, axis=0)
print(img.shape)

d = {}
d[inputs[0].name] = img

output_det = sess.run(None, d)

NodeArg(name='x', type='tensor(float)', shape=['DynamicDimension.0', 3, 'DynamicDimension.1', 'DynamicDimension.2']) x <class 'str'>
(640, 640, 3)
image padded (640, 640, 3)
expected img shape (640, 640, 3)
transposed img (3, 640, 640)
(1, 3, 640, 640)


In [146]:
maps = output_det[0]
print(maps.shape)

pred = maps[:, 0, :, :]
print(pred.shape)

thres = 0.3
# binarization
segmentation = pred > thres
print(segmentation.shape)

idx = 0
w, h, _ = raw_img.shape
mask = segmentation[idx]
print("mask sum",  np.sum(mask), mask.shape)

contours, heirarchy = cv2.findContours(
    (mask * 255).astype(np.uint8), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE
)
# print(contours)

boxes = []
merged_boxes = []

print(padded_img.shape, raw_img.shape)

# naive merge 
# TODO: need to understand the rotated rectangles edge case according to gemini
for index in range(len(contours)):
    contour = contours[index]
    
    points, sside = get_mini_boxes(contour)
    
    if len(merged_boxes) > 0:
        x_val_1, y_val_1 = min_x_y_point(points)
        x_val_2, y_val_2 = min_x_y_point(merged_boxes[-1])

        min_x_distance = min(abs(x_val_1[0] - x_val_2[1]), abs(x_val_2[0] - x_val_1[1]))
        min_y_distance = min(abs(y_val_1[0] - y_val_2[1]), abs(y_val_2[0] - y_val_1[1]))
        
        # print(index, min_x_distance, min_y_distance)
        if min_x_distance <= 20 and min_y_distance <= 15:
            print("merged", min_x_distance, min_y_distance)
            merged_points, _ = get_mini_boxes(np.concatenate([merged_boxes[-1], points]))
            merged_boxes[-1] = merged_points
            continue
            
    merged_boxes.append(points)

for index in range(len(merged_boxes)):
    contour = merged_boxes[index]
    # print("contour", contour)
    points, sside = get_mini_boxes(contour)
    # print("points", points)
        
    box = unclip(points, 2.0)
    print("points", box)

    if not box or not len(box):
        print("invalid box", points, box)
        continue
    
    box = np.array(box).reshape(-1, 1, 2)
    box, sside = get_mini_boxes(box)
    
    # box[:, 0] = np.clip(np.round(box[:, 0] / padded_img.shape[1] * raw_img.shape[1]), 0, raw_img.shape[1])
    # box[:, 1] = np.clip(
    #     np.round(box[:, 1] / padded_img.shape[0] * raw_img.shape[0]), 0, raw_img.shape[0]
    # )

    boxes.append(box.astype("int32"))

boxxed_img = draw_text_det_res(boxes, raw_img)
cv2.imwrite("det_output.png", boxxed_img)
    
img_with_cont = cv2.drawContours(padded_img, contours, -1, (0,255,0), 3)
cv2.imwrite("unprocessed_contours_det_output.png", img_with_cont)

(1, 1, 640, 640)
(1, 640, 640)
(1, 640, 640)
mask sum 15416 (640, 640)
(640, 640, 3) (640, 640, 3)
merged 2.0000305 2.999939
points [[[526, 484], [530, 485], [532, 488], [533, 492], [532, 499], [530, 502], [527, 505], [525, 505], [436, 504], [432, 503], [430, 500], [429, 496], [430, 488], [431, 485], [434, 482], [437, 482]]]
points [[[316, 468], [321, 469], [324, 472], [327, 476], [327, 480], [326, 492], [325, 496], [322, 500], [317, 502], [315, 502], [136, 500], [131, 499], [128, 496], [125, 492], [125, 488], [126, 477], [127, 473], [131, 469], [137, 467]]]
points [[[163, 442], [164, 444], [164, 449], [163, 451], [161, 452], [155, 452], [153, 451], [152, 449], [152, 444], [153, 442], [155, 441], [161, 441]]]
points [[[518, 422], [525, 423], [530, 426], [535, 431], [537, 436], [538, 443], [537, 470], [536, 476], [533, 481], [528, 486], [522, 488], [516, 489], [435, 486], [429, 485], [424, 482], [419, 477], [417, 471], [416, 465], [417, 439], [418, 433], [421, 428], [426, 423], [432, 42

True

In [147]:
def get_rotate_crop_image(img, points):
    img_crop_width = int(
        max(
            np.linalg.norm(points[0] - points[1]), np.linalg.norm(points[2] - points[3])
        )
    )
    img_crop_height = int(
        max(
            np.linalg.norm(points[0] - points[3]), np.linalg.norm(points[1] - points[2])
        )
    )

    # print("crop_dims", img_crop_width, img_crop_height, points)
    
    pts_std = np.float32(
        [
            [0, 0],
            [img_crop_width, 0],
            [img_crop_width, img_crop_height],
            [0, img_crop_height],
        ]
    )
    M = cv2.getPerspectiveTransform(np.float32(points), pts_std)
    dst_img = cv2.warpPerspective(
        img,
        M,
        (img_crop_width, img_crop_height),
        borderMode=cv2.BORDER_REPLICATE,
        flags=cv2.INTER_CUBIC,
    )
    dst_img_height, dst_img_width = dst_img.shape[0:2]
    if dst_img_height * 1.0 / dst_img_width >= 1.5:
        dst_img = np.rot90(dst_img)
    return dst_img

def resize_norm_img(img, req_shape, req_ratio_wh):
    imgC, imgH, imgW = req_shape
    img_ratio = img.shape[1] / img.shape[0]
    resized_w = img.shape[1]     
    if math.ceil(imgH * img_ratio) > imgW:
        resized_w = imgW
    else:
        resized_w = int(math.ceil(imgH * img_ratio))

    resized_image = cv2.resize(img, (resized_w, imgH))
    resized_image = resized_image.astype("float32")
    resized_image = resized_image.transpose((2, 0, 1)) / 255
    resized_image -= 0.5
    resized_image /= 0.5

    padding_im = np.zeros((imgC, imgH, imgW), dtype=np.float32)
    padding_im[:, :, 0:resized_w] = resized_image

    return padding_im

In [148]:
crops = []

# make image crops using det-boxes
for bno in range(len(boxes)):
    bx = boxes[bno]
    img_crop = get_rotate_crop_image(raw_img, bx)
    cv2.imwrite(f"rec_crop_{bno}.png", img_crop)
    crops.append(img_crop)

rec_shape_required = (3, 48, 320) 
max_wh_ratio = 320 / 48

norm_crops = []
for cim in crops:
    norm_crop = resize_norm_img(cim, rec_shape_required, max_wh_ratio)
    norm_crop = np.expand_dims(norm_crop, axis = 0)
    norm_crops.append(norm_crop)
    
    # print("norm_crops", norm_crop.shape)
    
norm_crops = np.concatenate(norm_crops)
cv2.imwrite(f"rec_final_input.png", norm_crops[0].transpose(1, 2, 0))

sess = ort.InferenceSession(
                "../model/rec_onnx/model.onnx",
                providers=["CPUExecutionProvider"],
                sess_options=None,
            )
inputs = sess.get_inputs()

d = {}
d[inputs[0].name] = norm_crops

output_rec = sess.run(None, d)

In [149]:
preds = output_rec[0]

preds_idx = preds.argmax(axis=2)
preds_prob = preds.max(axis=2)

print(preds_idx, preds_prob)
    
character_file = "en_dict.txt"
characters = ["blank"]
with open(character_file, "rb") as fin:
    lines = fin.readlines()
    for line in lines:
        line = line.decode("utf-8").strip("\n").strip("\r\n")
        characters.append(line)
characters.append(" ")

inference_outputs = []

for idx in range(len(preds_idx)):
    selection = np.ones(len(preds_idx[idx]), dtype=bool)
    selection &= preds_idx[idx] != 0
    # print(preds_idx[idx][selection])
    char_list = [
        characters[text_id] for text_id in preds_idx[idx][selection]
    ]
    # print(char_list)
    inference_outputs.append(char_list)


[[ 0  0 36  0 37  0  0 64  0  0 31  0  0  0 18  0  0 37  0  0 38  0  0 35
   0  0 69  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 39  0  0 26  0 36  0  0 25  0  0 18  0  0  0 29 96 96  0  0 30  0
   0  0 18  0  0 29  0 26  0 28  0  0  0  0  0  0]
 [ 0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 39  0 26  0 36  0  0 18  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 25  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  0  0  7  0 94  0  0  3  0  0  3  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  7  0  0 94  0  0  2  0  0 10  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 21  0  0 30  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [150]:
print(inference_outputs)

expiry = None
card_number = None

for o in inference_outputs:
    nums_only = [x for x in o if x.isdigit() or x == "/"]
    space_only = [x for x in o if x == " "]
    
    slash_idx = -1
    try:
        slash_idx = nums_only.index("/")
    except ValueError:
        pass

    if slash_idx != -1 and slash_idx in [1, 2] and len(nums_only) - slash_idx - 1 == 2:
        if expiry is None or expiry < "".join(o):
            expiry = "".join(o)
    
    elif slash_idx != -1 and slash_idx == 1 and len(nums_only) - slash_idx - 1 == 2:
        if expiry is None or expiry < "0" + "".join(o):
            expiry = "0" + "".join(o)
            
    elif len(nums_only) > 10:
        if card_number and len(space_only) < 3:
            continue
            
        card_number = "".join([x for x in nums_only if x != "/"])

print(expiry)
print(card_number)

[['S', 'T', 'o', 'N', 'A', 'T', 'U', 'R', 't'], ['V', 'I', 'S', 'H', 'A', 'L', ' ', ' ', 'M', 'A', 'L', 'I', 'K'], ['1'], ['V', 'I', 'S', 'A'], ['H', '0'], ['0', '6', '/', '2', '2'], ['6', '/', '1', '9'], ['D', 'M'], ['4', '8', '5', '4', '9', '8', '3', '4', '5', '6', '7', '8'], ['w', 'o', 'r', 'l', 'd'], ['F', 'I', 'R', 'S', 'T'], ['W', 'e', ' ', 'u', 'n', 'd', 'e', 'e', 'r', 's', 's', 't', 'a', 'n', 'd', ' ', 'y', 'y', 'o', 'u', 'r', ' ', ' ', 'w', 'o', 'r', 'l', 'd'], ['R', 'E', 'G', 'A', 'L', 'I', 'A'], ['H', 'D', 'F', 'C', ' ', 'B', 'A', 'N', 'K']]
6/19
485498345678


In [151]:
character_file = "en_dict.txt"
characters = ["blank"]
with open(character_file, "rb") as fin:
    lines = fin.readlines()
    for line in lines:
        line = line.decode("utf-8").strip("\n").strip("\r\n")
        characters.append(line)
characters.append(" ")

print(len(characters))

97
